In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import scipy.spatial
import matplotlib.pyplot as plt

In [ ]:
# Load data; for test purposes, a dataframe with random entries is created here
data_all = pd.DataFrame(data=np.random.rand(100,5), columns=['AOD 123', 'AOD_534', 'AOD-501', 'flag', 'UTC'])
data_all['flag'] = np.array(data_all['flag'] < 0.5).astype('bool')
data_all.sort_values(by='UTC', inplace=True)

In [ ]:
data_all.head()

In [ ]:
# Defining subroutines to cloudscreening

# Getting list of columns with AOD
def get_AOD_cols(df):
    AOD_cols = [col for col in df.columns if col[:3] == 'AOD']
    return AOD_cols

# Getting array of wavelengths
def get_wavelengths(df):
    lambdas = [float(col[4:]) for col in df.columns if col[:3] == 'AOD']
    return lambdas

# Finding column name of reference wavelength
def get_ref_column(df, ref_wl):
    for col in df.columns:
        if col[:3] == 'AOD' and float(col[4:]) == ref_wl: name = col
    return name

# Calculating Angstrom parameters
def calc_angstrom(tau, lambdas):
    # tau in shape (n_time, n_wavelengths)
    # alpha is the linear coefficient, gamma the quadratic, beta is the constant term
    alpha = []
    beta = []
    gamma = []
    x = np.log(lambdas)
    y = np.log(tau)
    ntime, ntau = np.shape(tau)
    for k in range(ntime):
        if np.any(np.isnan(y[k,:])):
            alpha = np.append(alpha, np.nan)
            beta = np.append(beta, np.nan)
            gamma = np.append(gamma, np.nan)
        else:
            # Linear fit for alpha and beta
            fit_l = np.polynomial.polynomial.Polynomial.fit(x, y[k,:], 1)
            coeffs = fit_l.convert().coef # Convert to array of coefficients, index represents exponent
            alpha = np.append(alpha, -coeffs[1])
            beta = np.append(beta, np.exp(coeffs[0]))
            # Quadratic fit for gamma
            fit_q = np.polynomial.polynomial.Polynomial.fit(x, y[k,:], 2)
            coeffs = fit_q.convert().coef # Convert to array of coefficients, index represents exponent
            gamma = np.append(gamma, coeffs[2])
    return alpha, beta, gamma

# Calculating distance to nearest neighbours
def cluster_k_nearest(X_norm, flag, k):
    if np.sum(flag) > 0:
        kdt = scipy.spatial.cKDTree(X_norm[~flag, :]) # Establish tree on non-flagged datapoints only
    else:
        kdt = scipy.spatial.cKDTree(X_norm)
    dists, neighs = kdt.query(X_norm, k) # Calculate nearest neighbours - non-flagged points will have themselves as 1st NN
    avg_dists = np.nanmean(dists, axis=1)
    return avg_dists

In [ ]:
# Cloudscreening routine according to a nearest neighbour algorithm
# Input: 
# Dataframe with time in hours as float labelled 'UTC' and AOD values labelled as 'AOD?wavelength', 
# where ? is an arbitrary separator, and wavelength is given in units of nm, e.g. 'AOD 501' or 'AOD_1020'
# If data is preprocessed and includes low quality, flag refers to the respective quality column (bool)
# Optional: 
# reference_wavelength: Wavelength to be used as dimension in clustering, default 501
# scaling: Dictionary of floats by which dtau/dt, alpha, gamma gets divided by
#          the higher the number, the less weight the variable has
# k: Number of nearest neighbours to be used, default 20
# limit: If score exceeds limit, flagged as cloudy, default 0.012
# Returns:
# Boolean array whether point is cloudy or not
# If limit is set to None, the distance to nearest neighbours is returned

def cloudscreen(data, reference_lambda = 501, flag_label = None, 
                scaling = {'dtaudt': 12., 'alpha': 10., 'gamma': 20.}, k = 20, limit = 0.012):
    if flag_label:
        flag = data[flag_label]
    else:
        flag = []

    # Calculate angstrom parameters
    alpha, beta, gamma = calc_angstrom(data[get_AOD_cols(data)].values, get_wavelengths(data))
                
    if len(data) - np.sum(flag) < 5: # Too few datapoints for analysis
        scores = np.full(len(data), np.nan)
    else:
        # Set up dataframe for clustering algorithm
        X = pd.DataFrame(columns = ['AOD', 'dadt', 'alpha', 'gamma'])
        X['AOD'] = data[get_ref_column(data, reference_lambda)]
        X['dadt'] = data[get_ref_column(data, reference_lambda)].diff() / (scaling['dtaudt']*data['UTC'].diff())
        X['dadt'].iloc[0] = X['dadt'].iloc[1]
        X['alpha'] = alpha/scaling['alpha']
        X['gamma'] = gamma/scaling['gamma']

        # Turn into array of numbers
        X_norm = X.values

        # Mean distance is approximately proportional to k;
        # if number of points is less than originally intended (by set limit),
        # scale the mean distances accordingly
        # Can be removed if limit is adjusted anyway
        k_factor = 20./k

        # Between 5 and 20 possible neighbour points 
        # e.g. if screening operationally at the beginning of the day 
        # adapt k for nearest neighbour search
        if X_norm.shape[0] - np.sum(flag) < k: 
            k_ori = k
            k = X_norm.shape[0] - np.sum(flag)
            k_factor = k_ori/k

        scores = k_factor*cluster_k_nearest(X_norm, flag, k)

    if limit: return (scores > limit)
    else: return scores

In [ ]:
# Standard use
# Even if a flag column is present in the data, it
# will get ignore as long as 'flag_label' is not set
# Returns boolean array of same length as data_all
cloudflag = cloudscreen(data_all)

In [ ]:
# Customization of scaling, reference wavelength, nearest neighbours etc.
# Returns the mean distance to k nearest neighbours (setting limit = None)
distances = cloudscreen(data_all, reference_lambda = 123, flag_label = 'flag', 
                        scaling = {'dtaudt': 12., 'alpha': 10., 'gamma': 20.}, 
                        k = 10, limit = None)